# Overview

This notebook is used to train a matrix factorization model for recommendation.<br>
We'll consider the implicit features in the MovieLens100k dataset.<br>
We'll use tensorflow recommenders to achieve this.

## Import TFRS

First, install and import TFRS and needed packages

In [1]:
!pip install -q tensorflow_recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 4.7 MB/s eta 0:00:00


In [2]:
from typing import Dict, Text
import tensorflow as tf
import tensorflow_recommenders as tfrs
import urllib.request
import zipfile
import pandas as pd

In [3]:
SEED = 19011

In [ ]:
# python version: 3.10.11
tf.__version__, tfrs.__version__

('2.12.0', 'v0.7.3')

## Download and extract data

In [4]:
DATA_URL = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
filename = 'ml-latest-small.zip'

In [5]:
urllib.request.urlretrieve(DATA_URL, filename)

('ml-latest-small.zip', <http.client.HTTPMessage at 0x7fe145a2b6a0>)

In [6]:
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall()

## Load, prepare and split data

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv')

In [9]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [10]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [11]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [12]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [13]:
ratings = ratings.join(movies, on='movieId', lsuffix='', rsuffix='_', how='inner')[['userId', 'title', 'rating', 'timestamp']].rename(columns={'title':'movieTitle'})
ratings

,userId,movieTitle,rating,timestamp
0,1,Jumanji (1995),4.0,964982703
516,5,Jumanji (1995),4.0,847434962
874,7,Jumanji (1995),4.5,1106635946
1434,15,Jumanji (1995),2.5,1510577970
1667,17,Jumanji (1995),4.5,1305696483
...,...,...,...,...
99945,610,Mrs. Henderson Presents (2005),3.5,1479542444
100012,610,Planet 51 (2009),3.0,1493848602
100033,610,Source Code (2011),2.5,1479544865
100038,610,"Master, The (2012)",4.0,1495959169


In [14]:
movies = movies.rename(columns={'title':'movieTitle'})
movies

,movieId,movieTitle,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [15]:
ratings['userId'] = ratings['userId'].map(lambda id_int: str(id_int))
movies['movieId'] = movies['movieId'].map(lambda id_int: str(id_int))

In [16]:
train_valid , test = train_test_split(ratings, test_size=0.2, stratify=ratings['userId'], random_state=SEED)

In [17]:
train, valid = train_test_split(train_valid, test_size=0.1, stratify=train_valid['userId'], random_state=SEED)

In [18]:
train.head()

,userId,movieTitle,rating,timestamp
31264,217,I Saw What You Did (1965),2.0,955942252
36013,246,Twin Falls Idaho (1999),4.0,1353864835
8562,58,Four Weddings and a Funeral (1994),5.0,847718851
93179,599,Marnie (1964),3.0,1498521686
94007,599,Anna Christie (1930),2.5,1498515351


### Cold Start Problem

For the cold start problem (new users with no history or guests with no accounts), we'll use aggregates about the movies to show the highest rated movies and most viewed movies (since we don't have the count of views, we'll use the count of ratings instead)


We'll create a custom class to handle this.<br>
We'll use thresholds to weed out movies with few ratings and movies with low ratings

In [19]:
class MovieData:
  def __init__(self, data, rating_threshold, count_threshold):
    self.rating_threshold = rating_threshold
    self.count_threshold = count_threshold
    self.data = data

  def get_highest_rated(self, n=20):
    # Return top n rated movies rated at least self.count_threhold times
    ratings_count = self.data.groupby(['movieTitle'])['rating'].count()
    popular_movies = ratings_count[ratings_count>self.count_threshold].index
    highest_rated_movies = self.data[self.data['movieTitle'].isin(popular_movies)].groupby('movieTitle').mean('rating')['rating'].sort_values(ascending=False)[:n]
    return highest_rated_movies

  def get_most_rated(self, n=20):
    # Return top n most rated movies with average rating more than self.rating_threhold times
    average_rating = self.data.groupby(['movieTitle'])['rating'].mean('rating')
    popular_movies = average_rating[average_rating>self.rating_threshold].index
    most_rated_movies = self.data[self.data['movieTitle'].isin(popular_movies)].groupby('movieTitle').count()['userId'].sort_values(ascending=False)[:n]
    return most_rated_movies

## Data Preparation

We'll create a tf dataset object for our train and test sets

In [20]:
train_interaction_dataset = tf.data.Dataset.from_tensor_slices({'userId':train['userId'].values, 'movieTitle': train['movieTitle'].values})
valid_interaction_dataset = tf.data.Dataset.from_tensor_slices({'userId':valid['userId'].values, 'movieTitle': valid['movieTitle'].values})
test_interaction_dataset = tf.data.Dataset.from_tensor_slices({'userId':test['userId'].values, 'movieTitle': test['movieTitle'].values})
train_interaction_dataset

<_TensorSliceDataset element_spec={'userId': TensorSpec(shape=(), dtype=tf.string, name=None), 'movieTitle': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [21]:
movie_dataset = tf.data.Dataset.from_tensor_slices(movies['movieTitle'].values)
movie_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [22]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None, name='users_lookup')
movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None, name='movies_lookup')

In [23]:
user_ids_vocabulary.adapt(train_interaction_dataset.map(lambda x: x['userId']))

In [24]:
movie_titles_vocabulary.adapt(movie_dataset.map(lambda x: x))

In [25]:
n_users = user_ids_vocabulary.vocabulary_size()
n_movies = movie_titles_vocabulary.vocabulary_size()
n_users, n_movies

(611, 9738)

## Define a model
We will use matrix factorization model without context features.
We can define a TFRS model by inheriting from `tfrs.Model` and implementing the `compute_loss` method:

The task is a convenient object that wraps both the loss and the metrics

In [26]:
class MovieLensModel(tfrs.Model):
  def __init__(self, user_model: tf.keras.Model, movie_model: tf.keras.Model, task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.
    user_embeddings = self.user_model(features["userId"])
    movie_embeddings = self.movie_model(features["movieTitle"])
    return self.task(user_embeddings, movie_embeddings)

Define the two models and the retrieval task.

In [27]:
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(n_movies, 64, name='movie_embedding')
], name='movie_model')

In [28]:
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(n_users, 64, name='user_embedding')
], name='user_model')

ks is the k for top_k metrics. We use multiple ks

In [29]:
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    candidates=movie_dataset.batch(128).map(movie_model),
    ks = (1, 5, 10)
  )
)

## Fit and evaluate it.

Create the model, train it, and generate predictions:



In [30]:
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.01))

In [ ]:
model.fit(train_interaction_dataset.batch(4096), epochs=15, validation_data=valid_interaction_dataset.batch(1024))

In [33]:
model.evaluate(test_interaction_dataset.batch(1024))

16/16 [==============================] - 6s 322ms/step - factorized_top_k/top_1_categorical_accuracy: 4.4906e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0106 - factorized_top_k/top_10_categorical_accuracy: 0.0238 - loss: 6234.4610 - regularization_loss: 0.0000e+00 - total_loss: 6234.4610


[0.000449063372798264,
 0.010649217292666435,
 0.023800360038876534,
 1193.6904296875,
 0,
 1193.6904296875]

## Indexers

Indexers use store the embedding of the possible candidates as keys. When it receives a query, it embeds the query and retrieves the closest keys.

For our recommendation task, it stores the embeddings of movies and the embedding of users. When we want to recommend for a user, it gets the movies whose embedding are the most similar (using dot product) to the user.

In [34]:
# Use brute-force search to set up retrieval using the trained representations.
user_recommender = tfrs.layers.factorized_top_k.BruteForce(model.user_model, k=100)

In [35]:
user_recommender.index_from_dataset(
    movie_dataset.batch(100).map(lambda title: (title, model.movie_model(title))))

In [36]:
# Get some recommendations.
_, titles = user_recommender(tf.constant(["42"]))
print(f"Top 3 recommendations for user 42: {titles.shape}")

Top 3 recommendations for user 42: (1, 100)


#### Item-Item recommendation

For items similarity, we can use the embedding of movies as both query and keys

In [37]:
movie_recommender = tfrs.layers.factorized_top_k.BruteForce(model.movie_model, k=100)

In [38]:
movie_recommender.index_from_dataset(
    movie_dataset.batch(100).map(lambda title: (title, model.movie_model(title))))

In [39]:
# Get some recommendations.
_, titles2 = movie_recommender(tf.constant(["Freaky Friday (2003)"]))
print(f"Top 3 recommendations for movie 42: {titles2.shape}")

Top 3 recommendations for movie 42: (1, 100)


In [40]:
# Get some recommendations.
_, titles2 = movie_recommender(tf.constant(["Freaky Friday (2003)"]), k=25)
print(f"Top 3 recommendations for movie 42: {titles2.shape}")

Top 3 recommendations for movie 42: (1, 25)


## Saving the models

In [41]:
user_recommender.save('user_model')
movie_recommender.save('movie_model')

In [42]:
tmp1 = tf.keras.models.load_model('user_model')
tmp2 = tf.keras.models.load_model('movie_model')

In [43]:
# Get some recommendations.
_, titles = tmp1(tf.constant(["42"]))
print(f"Top 3 recommendations for user 42: {titles.shape}")

Top 3 recommendations for user 42: (1, 100)


In [44]:
# Get some recommendations.
_, titles2 = tmp2(tf.constant(["Freaky Friday (2003)"]))
print(f"Top 3 recommendations for movie 42: {titles2[0, :10]}")

Top 3 recommendations for movie 42: [b'Dracula: Dead and Loving It (1995)' b'Little Women (1994)'
 b'Natural Born Killers (1994)' b'Man of the House (1995)' b'Safe (1995)'
 b'Boys of St. Vincent, The (1992)' b'Copycat (1995)'
 b'Street Fighter (1994)' b'North (1994)'
 b'New York Cop (Ny\xc3\xbb Y\xc3\xb4ku no koppu) (1993)']


# Explicit rating

## Data Prepatation

In [45]:
train_rating_dataset = tf.data.Dataset.from_tensor_slices({'userId':train['userId'].values, 'movieTitle': train['movieTitle'].values, 'rating': train['rating'].values})
valid_rating_dataset = tf.data.Dataset.from_tensor_slices({'userId':valid['userId'].values, 'movieTitle': valid['movieTitle'].values, 'rating': valid['rating'].values})
test_rating_dataset = tf.data.Dataset.from_tensor_slices({'userId':test['userId'].values, 'movieTitle': test['movieTitle'].values, 'rating': test['rating'].values})
train_rating_dataset

<_TensorSliceDataset element_spec={'userId': TensorSpec(shape=(), dtype=tf.string, name=None), 'movieTitle': TensorSpec(shape=(), dtype=tf.string, name=None), 'rating': TensorSpec(shape=(), dtype=tf.float64, name=None)}>

In [46]:
# user_ids_vocabulary.adapt(train_rating_dataset.map(lambda x: x['userId']))

In [47]:
# movie_titles_vocabulary.adapt(movie_dataset.map(lambda x: x))

In [48]:
# n_users = user_ids_vocabulary.vocabulary_size()
# n_movies = movie_titles_vocabulary.vocabulary_size()
# n_users, n_movies

## Define model

In [49]:
ranking_task = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

In [50]:
rating_model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(64,  activation='relu'),
    tf.keras.layers.Dense(1)
], name='raing_model')

In [51]:
explicit_movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(n_movies, 64, name='movie_embedding')
], name='movie_model')

In [52]:
explicit_user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(n_users, 64, name='user_embedding')
], name='user_model')

In [53]:
class ExplicitMovieLensModel(tfrs.Model):
  def __init__(self, user_model: tf.keras.Model, movie_model: tf.keras.Model, rating_model:tf.keras.Model, task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

        # Compute predictions.
    self.rating_model = rating_model

    # Set up a ranking task.
    self.task = task

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    user_embeddings = self.user_model(features["userId"])
    movie_embeddings = self.movie_model(features["movieTitle"])
    return self.rating_model(tf.concat([user_embeddings, movie_embeddings], axis=1))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("rating")

    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

In [54]:
explicit_model = ExplicitMovieLensModel(user_model=explicit_user_model, movie_model=explicit_movie_model, rating_model=rating_model, task=ranking_task)
explicit_model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.05))

In [55]:
explicit_model.fit(train_rating_dataset.batch(4096), epochs=40, validation_data=valid_rating_dataset.batch(1024))

Epoch 1/40
14/14 [==============================] - 2s 55ms/step - root_mean_squared_error: 1.8356 - loss: 3.1728 - regularization_loss: 0.0000e+00 - total_loss: 3.1728 - val_root_mean_squared_error: 1.0491 - val_loss: 1.1184 - val_regularization_loss: 0.0000e+00 - val_total_loss: 1.1184
Epoch 2/40
14/14 [==============================] - 1s 49ms/step - root_mean_squared_error: 1.0398 - loss: 1.0796 - regularization_loss: 0.0000e+00 - total_loss: 1.0796 - val_root_mean_squared_error: 1.0321 - val_loss: 1.0817 - val_regularization_loss: 0.0000e+00 - val_total_loss: 1.0817
Epoch 3/40
14/14 [==============================] - 1s 53ms/step - root_mean_squared_error: 1.0235 - loss: 1.0459 - regularization_loss: 0.0000e+00 - total_loss: 1.0459 - val_root_mean_squared_error: 1.0166 - val_loss: 1.0397 - val_regularization_loss: 0.0000e+00 - val_total_loss: 1.0397
Epoch 4/40
14/14 [==============================] - 1s 49ms/step - root_mean_squared_error: 1.0065 - loss: 1.0112 - regularization_lo

In [56]:
explicit_model.evaluate(test_rating_dataset.batch(1024))

16/16 [==============================] - 0s 6ms/step - root_mean_squared_error: 0.8713 - loss: 0.7736 - regularization_loss: 0.0000e+00 - total_loss: 0.7736


[0.871284544467926, 0.897028923034668, 0, 0.897028923034668]

In [57]:
all_movies = movies['movieTitle'].unique().reshape(-1,1)

In [59]:
# Predict rating for all movies
preds = explicit_model({"userId": tf.tile([['42']], [9737, 1]), "movieTitle": all_movies})
preds

<tf.Tensor: shape=(9737, 1), dtype=float32, numpy=
array([[3.4965222],
       [4.0983405],
       [3.6603467],
       ...,
       [3.8207724],
       [3.7071478],
       [3.7542446]], dtype=float32)>

In [60]:
# Sort movie titles from highest rated to lowest
tf.gather(all_movies, tf.squeeze(tf.argsort(preds, axis=0, direction='DESCENDING')))

<tf.Tensor: shape=(9737, 1), dtype=string, numpy=
array([[b'Cement Garden, The (1993)'],
       [b'Leave It to Beaver (1997)'],
       [b'Pompatus of Love, The (1996)'],
       ...,
       [b'Savior (1998)'],
       [b'Cobb (1994)'],
       [b"Alice Doesn't Live Here Anymore (1974)"]], dtype=object)>

In [61]:
# save model
explicit_model.save('explicit_model')

In [62]:
explicit_loaded = tf.saved_model.load("explicit_model")

In [63]:
# Note: Saved model takes input as 1-d array
preds = explicit_loaded({"userId": tf.tile(['42'], [9737]), "movieTitle": movies['movieTitle'].unique()})
tf.gather(all_movies, tf.squeeze(tf.argsort(preds, axis=0, direction='DESCENDING'))).numpy()

array([[b'Cement Garden, The (1993)'],
       [b'Leave It to Beaver (1997)'],
       [b'Pompatus of Love, The (1996)'],
       ...,
       [b'Savior (1998)'],
       [b'Cobb (1994)'],
       [b"Alice Doesn't Live Here Anymore (1974)"]], dtype=object)

## Pushing

This section pushes the code and model to github

### Access token

Create an access token and store it

In [64]:
import getpass
TOKEN = getpass.getpass(prompt='Please enter your token: ')

Please enter your token: ··········


In [75]:
!git clone -b model_training https://$TOKEN@github.com/osamaoun97/MovieLens_Recommender_System.git

Cloning into 'MovieLens_Recommender_System'...
remote: Enumerating objects: 287, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 287 (delta 51), reused 90 (delta 32), pack-reused 145
Receiving objects: 100% (287/287), 45.40 MiB | 15.30 MiB/s, done.
Resolving deltas: 100% (104/104), done.


In [76]:
!cp -r /content/explicit_model /content/MovieLens_Recommender_System/models
!cp -r /content/user_model /content/MovieLens_Recommender_System/models
!cp -r /content/movie_model /content/MovieLens_Recommender_System/models

In [77]:
cd /content/MovieLens_Recommender_System

/content/MovieLens_Recommender_System


In [78]:
!python /content/MovieLens_Recommender_System/utils/user_rec_live.py

2023-06-21 16:51:11.211634: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
recommendations: [b'Cement Garden, The (1993)' b'Leave It to Beaver (1997)'
 b'Pompatus of Love, The (1996)' ... b'Savior (1998)' b'Cobb (1994)'
 b"Alice Doesn't Live Here Anymore (1974)"]


In [ ]:
!git status

In [ ]:
!git add .

In [71]:
USER = 'AmgadHasan'
EMAIL = 'eng.amghasan@gmail.com'
!git config --global user.name $USER
!git config --global user.email $EMAIL

In [ ]:
!git commit -m "Retrained models after fixing the data spltting seed"

In [ ]:
!git status

In [ ]:
!git push origin model_training:model_training